In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

In [119]:
import re

In [3]:
cars_df = pd.read_csv("cars_dataset.csv", sep = "\t", encoding='utf-8')

In [5]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84174 entries, 0 to 84173
Data columns (total 34 columns):
url                        84174 non-null object
name                       84174 non-null object
model                      84174 non-null object
brand                      84174 non-null object
price                      84174 non-null object
eLabel                     84174 non-null object
bodyType                   83418 non-null object
length                     84174 non-null object
height                     84107 non-null object
width                      84140 non-null object
weight                     83899 non-null object
weightTotal                83899 non-null object
emissionsCO2               70163 non-null object
modelDate                  84174 non-null int64
fuelType                   83999 non-null object
numberOfAxles              84174 non-null int64
numberOfDoors              84174 non-null int64
numberOfForwardGears       84174 non-null object
seatingCapac

In [140]:
cars_df.iloc[0]

url                        http://www.cars-data.com//en/audi-s3-quattro-s...
name                                                         Audi S3 quattro
model                                                                Audi S3
brand                                                                   Audi
price                                                                 42.247
eLabel                                                                     G
bodyType                                                           hatchback
length                                                                  4159
height                                                               1415 mm
width                                                                1763 mm
weight                                                               1375 kg
weightTotal                                                          1375 kg
emissionsCO2                                                        218 g/km

In [141]:
num_cols = ['price', 'length', 'height', 'width', 'weight', 'weightTotal', 'emissionsCO2', 'numberOfAxles',
            'numberOfDoors', 'numberOfForwardGears', 'seatingCapacity', 'cargoVolume', 'roofLoad', 
            'accelerationTime', 'fuelCapacity', 'fuelConsumption', 'speed', 'payload', 'trailerWeight', 
            'vEengineDisplacement', 'vEenginePower', 'torque']

In [142]:
cat_cols = []
for col in cars_df.columns:
    if not (col in num_cols):
        cat_cols.append(col)

In [143]:
print(len(num_cols), len(cat_cols))
print(num_cols, '\n', cat_cols)

22 12
['price', 'length', 'height', 'width', 'weight', 'weightTotal', 'emissionsCO2', 'numberOfAxles', 'numberOfDoors', 'numberOfForwardGears', 'seatingCapacity', 'cargoVolume', 'roofLoad', 'accelerationTime', 'fuelCapacity', 'fuelConsumption', 'speed', 'payload', 'trailerWeight', 'vEengineDisplacement', 'vEenginePower', 'torque'] 
 ['url', 'name', 'model', 'brand', 'eLabel', 'bodyType', 'modelDate', 'fuelType', 'vehicleTransmission', 'driveWheelConfiguration', 'vEengineType', 'vEfuelType']


## Xử lý các cột dữ liệu số

In [144]:
# copy ra df để xư lý
df = cars_df.copy()

In [145]:
# Loại bỏ các cột không liên quan
df.drop(['url', 'name', 'model'], axis=1, inplace=True)

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84174 entries, 0 to 84173
Data columns (total 31 columns):
brand                      84174 non-null object
price                      84174 non-null object
eLabel                     84174 non-null object
bodyType                   83418 non-null object
length                     84174 non-null object
height                     84107 non-null object
width                      84140 non-null object
weight                     83899 non-null object
weightTotal                83899 non-null object
emissionsCO2               70163 non-null object
modelDate                  84174 non-null int64
fuelType                   83999 non-null object
numberOfAxles              84174 non-null int64
numberOfDoors              84174 non-null int64
numberOfForwardGears       84174 non-null object
seatingCapacity            84174 non-null object
vehicleTransmission        84174 non-null object
cargoVolume                84174 non-null object
roofLoad    

In [155]:
# fucntion convert cac thuoc tinh khac
def cvtFloat(x):
    if type(x) == str:
        temp = x.replace(',', '.').split()[0]
    else:
        temp = x
    val = None
    try:
        val = float(temp)
    except ValueError:
        return val
    return val

In [158]:
for el in num_cols:
    if el != 'cargoVolume':
        print(el)
        df[el] = df[el].apply(cvtFloat)

price
length
height
width
weight
weightTotal
emissionsCO2
numberOfAxles
numberOfDoors
numberOfForwardGears
seatingCapacity
roofLoad
accelerationTime
fuelCapacity
fuelConsumption
speed
payload
trailerWeight
vEengineDisplacement
vEenginePower
torque


In [170]:
def cvtFloat_cargoVolume(x):
    temp = x.split()[0]
    temp = temp.replace('-', ' ')
    temp = temp.split()
    if len(temp) > 0:
        temp = temp[-1]
    else:
        temp = x
    val = None
    try:
        val = float(temp)
    except ValueError:
        return val
    return val

In [172]:
df['cargoVolume'] = df['cargoVolume'].apply(cvtFloat_cargoVolume)

In [175]:
df.to_csv("cars_preprocessed.csv", sep = "\t", encoding='utf-8')

In [27]:
for cat in cat_cols[3:]:
    print(cat, len(cars_df[cat].unique()))

brand 89
eLabel 9
bodyType 11
modelDate 52
fuelType 11
driveWheelConfiguration 6
vEengineType 4
vEfuelType 11
